# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import requests

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries= []
# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        

# Print the city count to confirm sufficient count
list_of_cities = pd.DataFrame({"city" : cities})
list_of_cities.head()

,city
0,mataura
1,barrow
2,hilo
3,tsihombe
4,ushuaia


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
# Setting up values for OpenWeatherMap API calls
base_url = "http://api.openweathermap.org/data/2.5/weather"

app_id = api_key

params = { "appid" :app_id,"units":"Imperial" }
row_count = 0
for index,row in list_of_cities.iterrows():
    params["q"] =f'{row["city"]}'
    print(f"Processing Record {row_count} | {params['q']}")
    city_weather_resp = requests.get(base_url,params)
    print(city_weather_resp.url)
    city_weather_resp  = city_weather_resp.json()
    try:
        list_of_cities.set_value(index,"Cloudiness",city_weather_resp.get("clouds",{}).get("all"))
        list_of_cities.set_value(index,"Country",city_weather_resp.get("sys",{}).get("country"))
        list_of_cities.set_value(index,"Date",city_weather_resp.get("dt",{}))
        list_of_cities.set_value(index,"Humidity",city_weather_resp.get("main",{}).get("humidity"))
        list_of_cities.set_value(index,"Latitude",city_weather_resp.get("coord",{}).get("lat"))
        list_of_cities.set_value(index,"Longitude",city_weather_resp.get("coord",{}).get("lon"))
        list_of_cities.set_value(index,"Max Temperature",city_weather_resp.get("main",{}).get("temp_max"))
        list_of_cities.set_value(index,"Wind Speed",city_weather_resp.get("wind",{}).get("speed"))
        row_count = row_count + 1
    except(ValueError, KeyError , IndexError):
        print("missing result...... skiping to next city on list")
    

Processing Record 0 | mataura
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=mataura
Processing Record 1 | barrow


C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please

http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=barrow
Processing Record 2 | hilo
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=hilo
Processing Record 3 | tsihombe
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=tsihombe
missing result...... skiping to next city on list
Processing Record 3 | ushuaia
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=ushuaia
Processing Record 4 | saldanha
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=saldanha
Processing Record 5 | ponta do sol
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&units=Imperial&q=ponta+do+sol
Processing Record 6 | ribeira grande
http://api.openweathermap.org/data/2.5/weather?appid=19b32067ed32e919fafe0c91f8bf0faa&unit

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [4]:
list_of_cities =list_of_cities.dropna()
list_of_cities.reset_index(drop=True, inplace=True)
list_of_cities['Date'] = list_of_cities["Date"].astype('int64', copy=False)
list_of_cities.to_csv("City_Weather_data.csv")
list_of_cities

,city,Cloudiness,Country,Date


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [5]:
# Plot settings
def set_plot_prop(x_title,x_lim,y_title):
    plt.title(f"{x_title} vs {y_title}")
    plt.ylabel(y_title)
    plt.xlabel(x_title)
    plt.grid(True)
    plt.xlim(x_lim)

#### Latitude vs. Temperature Plot

In [6]:
list_of_cities.plot(kind="scatter",x="Latitude",y="Max Temperature",grid=True,color="blue")
set_plot_prop("Latitude",[-90,90],"Temperature (F)")
plt.axvline(0, color='black',alpha=0.5)
plt.savefig("Latitude vs Temperature")
plt.show()

KeyError: 'Latitude'

#### Latitude vs. Humidity Plot

In [ ]:
list_of_cities.plot(kind="scatter",x="Latitude",y="Humidity",grid=True,color="blue")
set_plot_prop("Latitude",[-90,90],"Humidity")
plt.axvline(0, color='black',alpha=0.5)
plt.savefig("Latitude vs Humidity")
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
list_of_cities.plot(kind="scatter",x="Latitude",y="Cloudiness",grid=True,color="blue")
set_plot_prop("Latitude",[-90,90],"Cloudiness")
plt.axvline(0, color='black',alpha=0.5)
plt.savefig("Latitude vs Cloudiness")
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
list_of_cities.plot(kind="scatter",x="Latitude",y="Wind Speed",grid=True,color="blue")
set_plot_prop("Latitude",[-90,90],"Wind Speed")
plt.axvline(0, color='black',alpha=0.5)
plt.savefig("Latitude vs Wind Speed")
plt.show()